This is a demonstration of how the datasets could be utilized to compute Residual Volatility (IVOL) factor ranks

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import re
import json
import requests
from yahooquery import Ticker
import numpy as np

directory = os.fsencode('../data')
res_series = []
beta_series = []

for file in os.scandir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.json'):
        try:
            with open(filename) as f:
                index = re.search('(?<=-).+(?= )', f.name).group()  # extract date
                data = json.load(f)
                # backwards compatiability for data sets that didn't have residuals calculated
                if 'residuals' in data:
                    res_json_series = pd.Series(data['residuals'])
                    beta_json_series = pd.Series(data['values'])
                    res_series.append(res_json_series.rename(index))
                    beta_series.append(beta_json_series.rename(index))
        except ValueError:
            continue

res_df = pd.DataFrame(res_series).sort_index()
res_df.index = pd.to_datetime(res_df.index)
betas_df = pd.DataFrame(beta_series).sort_index()
betas_df.index = pd.to_datetime(betas_df.index)

df_monthly = res_df.resample('M').std().dropna(axis='index', how='all') # convert to IVOL
upper_ivols = df_monthly.apply(lambda x: x[x >= x.quantile(.8)], axis=1) # upper quintile IVOL
bottom_ivols = df_monthly.apply(lambda x: x[x <= x.quantile(.25)], axis=1) # bottom quintile IVOL

df_monthly

,A,AA,AADI,AAL,AAME,AAOI,AAON,AAP,AAPL,AB,...,YQ,YRD,YSG,YY,ZH,ZK,ZKH,ZLAB,ZTO,EMP
2024-01-31,NaN,0.000254,0.000221,0.000131,0.000205,0.000533,0.000263,0.000219,0.000113,0.000101,...,0.000355,0.000383,0.000227,0.000376,0.000154,NaN,NaN,0.000340,0.000352,NaN
2024-02-29,NaN,0.000209,0.000174,0.000119,0.000286,0.000194,0.000147,0.000299,0.000114,0.000080,...,0.000258,0.000143,0.000603,0.000434,0.000177,NaN,NaN,0.000248,0.000324,NaN
2024-03-31,NaN,0.000425,0.000263,0.000265,0.000229,0.000171,0.000243,0.000398,0.000156,0.000181,...,0.000212,0.000196,0.000392,0.000185,0.000164,NaN,NaN,0.000460,0.000260,NaN
2024-04-30,NaN,0.000322,0.000331,0.000230,0.000411,0.000416,0.000268,0.000349,0.000199,0.000215,...,0.000347,0.000192,0.000661,0.000174,0.000355,NaN,NaN,0.000289,0.000238,NaN
2024-05-31,NaN,0.000168,0.000157,0.000268,0.000203,0.000234,0.000275,0.000185,0.000129,0.000117,...,0.000318,0.000183,0.000152,0.000345,0.000391,NaN,NaN,0.000282,0.000187,NaN
2024-06-30,0.000079,0.000190,0.000198,0.000101,0.000142,0.000089,0.000304,0.000118,0.000099,0.000140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-31,0.000549,0.000533,0.000283,0.000224,0.000281,0.000527,0.000265,0.000275,0.000169,0.000292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-31,0.000144,0.000158,0.000379,0.000131,0.000178,0.000398,0.000158,0.000379,0.000083,0.000226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-30,0.000088,0.000332,0.000136,0.000256,0.000240,0.000439,0.000392,0.000171,0.000093,0.000059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-31,0.000070,0.000409,0.000203,0.000412,0.000108,0.000122,0.000144,0.000180,0.000106,0.000362,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now I'll scrape CBOE weekly options CSV for tickers and intersect them with the top and bottom quartiles of the IVOL portfolio

In [2]:
cboe_csv_link = 'https://www.cboe.com/available_weeklys/get_csv_download/'
output = requests.get(cboe_csv_link).text

find_str = "Available Weeklys - Exchange Traded Products (ETFs and ETNs)"

idx = output.find(find_str)

skiprows_val = output[:idx+len(find_str)].count("\n")


cboe_csv = pd.read_csv(cboe_csv_link, skiprows=skiprows_val, usecols=[0], header=None)
tickers_df = cboe_csv[(cboe_csv[0] != 'Available Weeklys - Exchange Traded Products (ETFs and ETNs)')
                      & (cboe_csv[0] != 'Available Weeklys - Equity')]

weekly_option_tickers = tickers_df[0]

recent_upper_ivols = upper_ivols.iloc[-1]
upper_ivol_tickers = recent_upper_ivols[recent_upper_ivols.notna()].index
recent_bottom_ivols = bottom_ivols.iloc[-1]
bottom_ivol_tickers = recent_bottom_ivols[recent_bottom_ivols.notna()].index

optionable_upper_ivol_tickers = weekly_option_tickers[weekly_option_tickers.isin(upper_ivol_tickers)]
optionable_bottom_ivol_tickers = weekly_option_tickers[weekly_option_tickers.isin(bottom_ivol_tickers)]

Now, I want to intersect further with the lower quartile of the low beta portfolio from the recent month.

In [3]:
low_beta = betas_df.resample('M').mean().apply(lambda x: x[x <= x.quantile(.25)], axis=1)
recent_low_betas = low_beta.iloc[-1]
low_beta_tickers = recent_low_betas[recent_low_betas.notna()].index
optionable_bottom_ivol_tickers[optionable_bottom_ivol_tickers.isin(low_beta_tickers)]

110     ADP
170     BSX
179     CAG
180     CAH
186    CBOE
195    CHTR
197      CI
199      CL
204     CLX
215     CPB
217    CPRI
230     CVX
242      DG
253     DPZ
257      EA
268     EPD
270      ET
284    FOXA
298    GILD
325     HRL
331     IBM
353       K
358     KMB
371     LMT
389     MCK
391    MDLZ
402      MO
406     MRK
428     NOC
460     PEP
462      PG
469      PM
493     RTX
530     STZ
534       T
553    TSCO
567     UNH
575       V
597     WMT
Name: 0, dtype: object

These are all presumably safe stocks to long. I could create carry by short selling puts, trade their skew, or even make a dispersion trade by shorting options against the opposite leg of the IVOL factor.

Next, let's create a factor based on US presedential election odds, then test how it correlates with our residuals 


In [6]:
rawdata = '../data/2024ElectionOdds.csv'
odds = 1/pd.read_csv(rawdata,index_col=0)
trump_odds_changes = odds['Donald Trump'].pct_change()
trump_odds_changes.index = pd.to_datetime(trump_odds_changes.index, dayfirst=True)

weekly_res_df = res_df.resample('W').sum(min_count=1).iloc[-21:].dropna(axis=1) # Starting ~June
weekly_aligned_trump_factor = trump_odds_changes.reindex(weekly_res_df.index, method='ffill')
joined_df = weekly_res_df.join(weekly_aligned_trump_factor.rename('trump_factor'))

trump_factor_corr = joined_df.drop(columns=['trump_factor']).corrwith(joined_df['trump_factor'], method='pearson')


In [7]:
with pd.option_context('display.max_rows',0):
    display(trump_factor_corr[trump_factor_corr.abs() > .5].sort_values())

BKD    -0.696745
SPOK   -0.693601
KBH    -0.669924
CCS    -0.661011
SCI    -0.643941
ALEX   -0.624880
BHRB   -0.622458
CPK    -0.619752
USPH   -0.619263
EVG    -0.616065
KFY    -0.615766
SRDX   -0.614664
          ...   
HOLO    0.562396
QRHC    0.563925
AWX     0.565584
INVZ    0.585516
BZ      0.586907
NTAP    0.592213
TT      0.598758
LIDR    0.609199
SHOP    0.623697
CRDO    0.635809
VTEX    0.649093
ENVX    0.656385
Length: 160, dtype: float64